In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(
    model="gpt-5.2",
    temperature=0.2,
    max_tokens=1000,
    timeout=30
)

# Router template: outputs list of agents
router_template = ChatPromptTemplate.from_messages([
("system", """
You are an intelligent router in a multi-agent system
Your task is to read a user question and determine which agents can answer it. 
Output MUST be a Python list of agent names as strings. 
Do NOT give any explanation, reasoning, or extra text.
"""),

("human", """
Available agents:
- customer: contains customer details (IDs, locations)
- seller: contains seller details (IDs, locations)
- orders: contains order details (product IDs, order IDs, items, prices, delivery, payment)
- product: contains product details (ID, category, description, dimensions)

Instructions:
1. Break the question into subquestions.
2. For each subquestion, decide which agents have the required information.
3. At the end, return a Python list of all agents needed for the full question.

Examples:
- If customer and orders agents are needed: ['customer', 'orders']
- If only customer is needed: ['customer']
- If all four are needed: ['customer', 'seller', 'orders', 'product']

User question:
{question}
""")
])

router_chain = router_template | model | StrOutputParser()

def router_agent(user_question):
    """
    Returns a list of agent names that should handle the question
    """
    response = router_chain.invoke({"question": user_question})
    # Convert string output to actual Python list safely
    try:
        response = eval(response)
        if isinstance(response, list):
            print(response)
            return response
        return []
    except Exception:
        return []


In [24]:
def customer_agent(query):
    return f"Customer agent handled query: {query}"

def order_agent(query):
    return f"Orders agent handled query: {query}"

def product_agent(query):
    return f"Product agent handled query: {query}"

def seller_agent(query):
    return f"Seller agent handled query: {query}"


def handle_user_query(query):
    selected_agents = router_agent(query)
    results = []

    for agent_name in selected_agents:
        if agent_name.lower() == "customer":
            results.append(customer_agent(query))
        elif agent_name.lower() == "seller":
            results.append(seller_agent(query))
        elif agent_name.lower() == "order":
            results.append(orders_agent(query))
        elif agent_name.lower() == "product":
            results.append(product_agent(query))
    
    return results


In [26]:
query = "Show all orders and mode of payment by customers in Berlin "

responses = handle_user_query(query)
for r in responses:
    print(r)


['customer', 'orders']
Customer agent handled query: Show all orders and mode of payment by customers in Berlin 
